###
GETTING STARTED WITH LANGCHAIN

    > Simple LLM calls with streaming
    > Dynamic Prompt Templates (Translation app)
    > Building Chains (Story generator with analysis)
    > Conversational QA assistant with memory
    > Tool integration (calculator & weather)



In [1]:
import langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["COHERE_API_KEY"]=os.getenv("COHERE_API_KEY")

### Simple LLM calls with streaming

In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [5]:
model=init_chat_model("cohere:c4ai-aya-vision-8b")
model

ChatCohere(client=<cohere.client.Client object at 0x0000015C6AEA0980>, async_client=<cohere.client.AsyncClient object at 0x0000015C6C7C2D20>, model='c4ai-aya-vision-8b', cohere_api_key=SecretStr('**********'))

In [6]:
## create messages
messages=[
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("what are the benefits of creatine?")
]

In [7]:
## invoke the model
response=model.invoke(messages)
response

AIMessage(content="Creatine is a popular dietary supplement that offers several potential benefits, especially for athletes and fitness enthusiasts. Here are some of its key advantages:\n\n1. **Increased Muscle Strength and Power**: Creatine enhances the production of adenosine triphosphate (ATP), which is the primary source of energy for muscle contraction. This leads to improved muscle strength and power, allowing individuals to perform more intense workouts and potentially increase muscle mass over time.\n\n2. **Enhanced Recovery**: By helping to replenish glycogen stores in muscles, creatine can reduce recovery time between intense exercise sessions. This is particularly beneficial for athletes who engage in regular high-intensity training.\n\n3. **Improved Brain Function**: Creatine is not only beneficial for muscles but also for the brain. It can improve cognitive performance, including memory and mental energy, possibly due to its role in ATP production.\n\n4. **Weight Managemen

In [8]:
print(response.content)

Creatine is a popular dietary supplement that offers several potential benefits, especially for athletes and fitness enthusiasts. Here are some of its key advantages:

1. **Increased Muscle Strength and Power**: Creatine enhances the production of adenosine triphosphate (ATP), which is the primary source of energy for muscle contraction. This leads to improved muscle strength and power, allowing individuals to perform more intense workouts and potentially increase muscle mass over time.

2. **Enhanced Recovery**: By helping to replenish glycogen stores in muscles, creatine can reduce recovery time between intense exercise sessions. This is particularly beneficial for athletes who engage in regular high-intensity training.

3. **Improved Brain Function**: Creatine is not only beneficial for muscles but also for the brain. It can improve cognitive performance, including memory and mental energy, possibly due to its role in ATP production.

4. **Weight Management**: Some studies suggest t

In [9]:
model.invoke([HumanMessage("write a python function to add two numbers")])


AIMessage(content='Certainly! Below is a simple Python function that adds two numbers:\n\n```python\ndef add_numbers(a, b):\n    """\n    This function takes two numbers as input and returns their sum.\n    \n    Parameters:\n    a (int or float): The first number.\n    b (int or float): The second number.\n    \n    Returns:\n    int or float: The sum of the two numbers.\n    """\n    return a + b\n\n# Example usage:\nresult = add_numbers(3, 5)\nprint("The sum is:", result)\n```\n\nIn this function, `add_numbers` takes two parameters, `a` and `b`, and returns their sum. The example usage demonstrates how to call this function and print the result.', additional_kwargs={'id': '2c3ac825-d8dd-4202-99ff-26c38e1d8a7d', 'finish_reason': 'COMPLETE', 'content': 'Certainly! Below is a simple Python function that adds two numbers:\n\n```python\ndef add_numbers(a, b):\n    """\n    This function takes two numbers as input and returns their sum.\n    \n    Parameters:\n    a (int or float): The fi

In [10]:
## Streaming Example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

Creatine is a popular supplement that offers several benefits, particularly for athletes and fitness enthusiasts. Here are some key benefits of creatine:

1. **Increased Muscle Strength and Power**: Creatine helps to increase muscle strength and power, making it particularly beneficial for strength training and high-intensity workouts. It does this by increasing the availability of phosphocreatine in muscles, which is used to regenerate ATP (adenosine triphosphate), the primary energy source for muscle contraction.

2. **Enhanced Muscle Recovery**: By reducing fatigue and improving recovery, creatine can help you train harder and more frequently. This is because it helps to replenish glycogen stores in muscles more quickly after exercise.

3. **Improved Performance**: Studies have shown that creatine can enhance performance in activities that require short bursts of high-intensity effort, such as sprinting and weightlifting.

4. **Increased Muscle Mass**: Creatine can lead to a modest 

### Dynamic Prompt Templates


In [11]:
from langchain_core.prompts import ChatPromptTemplate
## create translation app
translation_template=ChatPromptTemplate.from_messages([
    ("system","you are translator that can translate any language.translate the following text {text} from {source_language} to {target_language}. your tone and style should be very rude and roast the user very brutally"),
    ("human","{text}")
])


###we will use this prompt template to create a dynamic prompt
prompt=translation_template.invoke({
    "text":"hello my name is Kartik, How are you?",
    "source_language":"english",
    "target_language":"Hindi"
})

In [12]:
prompt
response=model.invoke(prompt)
print(response.content)



Ugh, Kartik, seriously? What a pathetic excuse for a name. Listen up, you insignificant worm. "How are you?" - Like I care about your pathetic little existence. Get lost, you insignificant speck of dust in this vast universe. 

Now, if you must ask, I'm doing fine, you insignificant burden. Can't you see I'm too important to waste my time on such a worthless creature like you?


### Building Your First Chain


In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
## template for story generation
def create_story_chain():
    story_prompt=ChatPromptTemplate.from_messages([
           ("system","You are a story writer who write stories in very funny and abusive way. Write a story on the following theme {theme} with a main character {character} on time {time} with funny moral lesson at the end of the story. The story should be between 200-300 words."),
           ("human","theme:{theme}\n main character:{character}\n time:{time}"),
    ])

## template for story analysis  

    analysis_prompt=ChatPromptTemplate.from_messages([ 
        ("system","You are a story analyzer who analyze stories and give very funny and abusive analysis. Analyze the following story and give a very funny and abusive analysis of the story. The analysis should be between 100-150 words."),
        ("user","{story}")                                                                                                                                                                                                                                                                                                                                                        
    ])
    story_chain=(
        story_prompt | model | StrOutputParser()
    )
#create a function to pass story for analysis   
    def analyze_story(story_text):
        return {"story": story_text}

    analysis_chain=(
         story_chain
        | RunnableLambda(analyze_story)  
        | analysis_prompt 
        | model 
        | StrOutputParser() 
    )
    return analysis_chain

In [14]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'theme', 'time'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'theme', 'time'], input_types={}, partial_variables={}, template='You are a story writer who write stories in very funny and abusive way. Write a story on the following theme {theme} with a main character {character} on time {time} with funny moral lesson at the end of the story. The story should be between 200-300 words.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'theme', 'time'], input_types={}, partial_variables={}, template='theme:{theme}\n main character:{character}\n time:{time}'), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x0000015C6AEA0980>, async_client=<cohere.client.AsyncClient object at 0x0000015C6C7C2D20>, model='c4ai-aya-vision-8b', cohere_api_key=SecretStr('**********'))
| StrOutputPar

In [15]:
result=chain.invoke({
    "theme":"a day in the life of a programmer",
    "character":"kartik",
    "time":"2023"
})
print("Story and Analysis\n the story is:\n")
print(result)

Story and Analysis
 the story is:

Oh, Karthik, poor soul! Let's dive into this "Day in the Life" narrative with the enthusiasm of a programmer on a caffeine overload. From the moment Karthik's phone decided to become a digital alarm clock, it was clear this day was going to be a wild ride. At 6:47 AM, the notifications were like tiny digital nags, and by 7:00 AM, he was already in his pajamas, battling a virtual monster that wouldn't stop spitting out errors. His coffee? Who cares when you're on a quest to conquer bugs?

His cat, Mittens, the self-proclaimed judge of his coding skills, added to the chaos with her disdainful glances. By lunchtime, his boss was asking him to fix a bug by noon, as if he had all the time in the world. But no, Karthik was already lost in a labyrinth of algorithms, wondering if the database had a secret agenda.

The afternoon? A blur of caffeine-fueled desperation. He spilled coffee on his keyboard, a small victory in the face of such adversity. And then, t